<a href="https://colab.research.google.com/github/lakshmivijay23/CLIP_similarity/blob/main/check_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision pillow transformers

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import torch
import torch.nn.functional as F
from PIL import Image

In [9]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

In [10]:
image_folder="/content/drive/MyDrive/images"

In [11]:
image_embeddings = {}
image_tensors = {}
for filename in os.listdir(image_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt", padding=True)
        with torch.no_grad():
            embedding = model.get_image_features(**inputs)
            embedding = embedding / embedding.norm(p=2, dim=-1, keepdim=True)  # normalize

        image_embeddings[filename] = embedding
        image_tensors[filename] = img

In [12]:
def cosine_similarity(a, b):
    return F.cosine_similarity(a, b).item()

In [13]:
def get_image_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt")

    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
        embedding = embedding / embedding.norm(p=2, dim=-1, keepdim=True)

    return embedding

In [14]:
def match_image(query_image_path, stored_embeddings):
    query_embedding = get_image_embedding(query_image_path)

    scores = {}
    for filename, emb in stored_embeddings.items():
        score = cosine_similarity(query_embedding, emb)
        scores[filename] = score

    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    best_match = next(iter(scores))
    best_score = scores[best_match]

    return best_match, best_score, scores

In [15]:
query_img = "/content/drive/MyDrive/test_images/condensing_unit_test.jpg"

best_match, best_score, all_scores = match_image(query_img, image_embeddings)
print(f"\nImage matched with: {best_match.split(".")[0].replace("_"," ").title()} - {best_score * 100:.2f}% similarity")


Image matched with: Condensing Unit - 85.85% similarity
